In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import random

In [2]:
metadata = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_metadata.tab",sep='\t')

In [3]:
metadata.head()

,#SampleID,BarcodeSequence,LinkerPrimerSequence,run_prefix,body_habitat,body_product,body_site,bodysite,dna_extracted,elevation,...,sample_type,scientific_name,sequencecenter,taxon_id,title,qiita_study_title,qiita_study_alias,qiita_owner,qiita_principal_investigator,Description
0,1927.SRS055168.SRX020678.SRR048038,TCAGACTCCTC,ATTACCGCGGCTGCTGG,SRR048038,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,R_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS055168.SRX020678.SRR048038
1,1927.SRS015346.SRX020577.SRR044088,TCAGAACCAGC,ATTACCGCGGCTGCTGG,SRR044088,UBERON:vagina,UBERON:mucus,UBERON:central vagina,Mid_vagina,True,97.0,...,Vaginal,human vaginal metagenome,WUGSC,1632839,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS015346.SRX020577.SRR044088
2,1927.SRS021613.SRX020675.SRR047757,TCAGAGCTTC,ATTACCGCGGCTGCTGG,SRR047757,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,Tongue_dorsum,True,97.0,...,Oral,human oral metagenome,BCM,447426,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS021613.SRX020675.SRR047757
3,1927.SRS022512.SRX020678.SRR048023,TCAGTATCAAC,ATTACCGCGGCTGCTGG,SRR048023,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,L_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS022512.SRX020678.SRR048023
4,1927.SRS016305.SRX020525.SRR044526,TCAGCCACGGTC,ATTACCGCGGCTGCTGG,SRR044526,UBERON:skin,UBERON:sebum,UBERON:skin of external ear,L_Retroauricular_crease,True,97.0,...,Skin,human skin metagenome,WUGSC,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS016305.SRX020525.SRR044526


In [4]:
data = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_data.tab",sep='\t')

In [5]:
data.head()

,#OTU ID,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,...,1928.SRS042698.SRX020530.SRR045847,1928.SRS023827.SRX021470.SRR052699,1928.SRS022769.SRX022226.SRR058089,1928.SRS024716.SRX021470.SRR052699,1928.SRS045191.SRX021470.SRR052699,1928.SRS045171.SRX021470.SRR052699,1928.SRS024704.SRX021470.SRR052699,1928.SRS020726.SRX022239.SRR058109,1928.SRS022067.SRX022231.SRR058120,1928.SRS023274.SRX022223.SRR058086
0,4376890,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,188753,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,114076,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,253429,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4467992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### extract relevant samples

In [6]:
filtered_data_with_OTUs = data.loc[:, data.sum() > 3000] #5728 samples, 13268 features
filtered_data = filtered_data_with_OTUs.iloc[:, 1:]
filtered_data

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### relative abundance - calc sum

In [7]:
final_data = filtered_data.copy()

for colname in filtered_data.columns:
    final_data.loc[:,colname] = filtered_data[colname] / sum(filtered_data[colname])

final_data.head()

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,0.001055,0.001894,0.000218,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000047,0.000213,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000248,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001515,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000164,0.00024,0.000265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Isolation Forest
data for IF

In [8]:
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
feces = [sample for sample in all_feces if sample in final_data.columns]
feces_data = final_data[feces] #418 samples

#all _body sites except of feces
all_body_sites = metadata[metadata['body_site'] != 'UBERON:feces' ]['body_site'].drop_duplicates()
print(all_body_sites)

0                    UBERON:skin of elbow
1                   UBERON:central vagina
2                           UBERON:tongue
4             UBERON:skin of external ear
5                            UBERON:mouth
6                            UBERON:tooth
8                  UBERON:palatine tonsil
11                   UBERON:buccal mucosa
14               UBERON:vaginal introitus
17                          UBERON:throat
18                         UBERON:gingiva
19                  UBERON:anterior nares
20                     UBERON:hard palate
64      UBERON:posterior fornix of vagina
3534                  UBERON:skin of nose
3537                        UBERON:saliva
Name: body_site, dtype: object


In [25]:
all_auc = []

for body_site in all_body_sites:
    now_auc = []
    i = 0
    while i < 50:
        all_body_site = metadata[metadata['body_site'] == body_site]['#SampleID']
        final_all_body_site = [sample for sample in all_body_site if sample in  final_data.columns]
        all_body_site_data = final_data[final_all_body_site]
        if len(all_body_site_data.iloc[0]) >= 42:
            indices = random.sample(range(0,len(all_body_site_data.iloc[0])), 42)
            few_all_body_site_data = all_body_site_data.iloc[: ,indices] #42 samples
            #few_all_body_site_data = all_body_site_data.iloc[: ,-42:] #42 samples
            more_res = [0 for i in range(42)]
            
        else:
            few_all_body_site_data = all_body_site_data
            more_res = [0 for i in range(len(few_all_body_site_data.iloc[0]))]
            
        results = [1 for i in range(418)]
        
        y = results + more_res

        inputIF = pd.concat([feces_data, few_all_body_site_data], axis=1) #42 samples out of 460 should be outliers

        model = IsolationForest(n_estimators=100)
        model.fit(inputIF.transpose())  # fit 100 trees

        scores = model.decision_function(inputIF.transpose()) #higher score means more normal
        #sortedScores = sorted(scores)
        # indices of anomalies (body site != feces) in sorted scores. higher score = higher indices = more normal !
        #anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:]]
        #print(anomalies_indices_in_sorted) # if algorithm is working ok, indices are expected to be low. in reality we get differernt indices, some are very high. 
        
        # calculate area under the curve
        now_auc.append(roc_auc_score(y, scores))
        i += 1
        
    all_auc.append(now_auc)
        
print(all_auc)

not_random_all_auc = [[0.616142629300524, 0.6383572567783093, 0.5963773069036227, 0.6338004101161995, 0.6377306903622694, 0.603611300979722, 0.5831624515835042, 0.6329175210754159, 0.7038049669628617, 0.5924755069491912, 0.6768626110731375, 0.5813966735019367, 0.6433128275233537, 0.6401230348598769, 0.6944064707222602, 0.6308384597858283, 0.6263385737069946, 0.5777511961722488, 0.6328320802005012, 0.6513442697653224, 0.6728753702437912, 0.6473000683526999, 0.5798587377534746, 0.5986557302346777, 0.5558213716108454, 0.6254272043745729, 0.6701982228298018, 0.7304055593529278, 0.6471861471861471, 0.6431989063568011, 0.7268740031897927, 0.6901913875598086, 0.5944976076555024, 0.697425381635908, 0.6835839598997494, 0.617680565048986, 0.7439052175894281, 0.6066871724766462, 0.5787764866712235, 0.5733082706766917, 0.6071428571428572, 0.6940077466393255, 0.578548644338118, 0.5864376851218956, 0.6512303485987696, 0.6529961266803372, 0.5143540669856459, 0.5881180223285487, 0.6483253588516746, 0.6507177033492824], [0.12141148325358851, 0.12494303941672366, 0.15789473684210525, 0.10042150831624516, 0.12263613579403057, 0.07379243563454091, 0.16547049441786285, 0.15880610617452723, 0.08965595807701071, 0.12152540442014127, 0.10856687172476648, 0.11506037821827296, 0.09822852586010482, 0.13793005240373662, 0.16569833675096837, 0.2314023695602643, 0.13727500569605833, 0.18571998177261334, 0.13189223057644112, 0.12012987012987016, 0.1386135794030531, 0.11335156071998179, 0.20243791296422875, 0.12388926862611073, 0.11369332421964004, 0.15017657780815677, 0.10608908635224426, 0.12642401458190933, 0.14035087719298245, 0.13915470494417864, 0.13140806561859195, 0.16410344041922992, 0.16250854408749146, 0.09022556390977443, 0.14205969469127364, 0.1494930508088403, 0.10084871269081797, 0.13738892686261106, 0.15117338801549327, 0.114149008885851, 0.12263613579403054, 0.12050011392116655, 0.15974595579858739, 0.13308840282524492, 0.19110275689223058, 0.08153907496012759, 0.11095921622237413, 0.1983937115516063, 0.19135907951697428, 0.09865573023467761], [0.4001480975165186, 0.406015037593985, 0.27665755297334244, 0.6065162907268171, 0.3470608339029392, 0.4154704944178629, 0.4258373205741627, 0.33014354066985646, 0.3967874231032126, 0.38345864661654133, 0.4993164730006835, 0.5464228753702438, 0.5902255639097744, 0.3761676919571656, 0.351617680565049, 0.6069150148097516, 0.5413533834586466, 0.47459557985873774, 0.4592731829573935, 0.4172932330827067, 0.41057188425609487, 0.3601617680565049, 0.33595352016404645, 0.4699248120300752, 0.5375939849624061, 0.5422647527910687, 0.4616085668717248, 0.3685919343814081, 0.39786967418546365, 0.38704716336295286, 0.4156983367509683, 0.5926748689906585, 0.6587491455912509, 0.3852244246981089, 0.36306675780359987, 0.5766689450899978, 0.5365686944634314, 0.39171793119161535, 0.44076099339257235, 0.45676691729323304, 0.4765322396901344, 0.32946001367054, 0.42116655274550013, 0.4336978810663021, 0.55622009569378, 0.4781840966051492, 0.5484164957849168, 0.36631351105035315, 0.4486785144679882, 0.3455798587377535], [0.37149692412850305, 0.26144907723855093, 0.29710640236956026, 0.28354978354978355, 0.25911369332421963, 0.2978468899521531, 0.3635794030530873, 0.3251310093415356, 0.23108908635224426, 0.44594440647072225, 0.36226930963773074, 0.30767259056732743, 0.2768284347231716, 0.33438710412394623, 0.2446172248803828, 0.24948735475051267, 0.20625427204374575, 0.28565732513100933, 0.31655844155844154, 0.2323137388926863, 0.3093814080656186, 0.21878560036454775, 0.39684438368648894, 0.28565732513100933, 0.32111528822055135, 0.27392344497607657, 0.2897015265436318, 0.2550125313283208, 0.2983025746183641, 0.23524720893141948, 0.23476304397357028, 0.199419002050581, 0.2774550011392117, 0.29892914103440416, 0.30451127819548873, 0.3212576896787423, 0.36426293005240373, 0.2845181134654819, 0.3364092048302575, 0.2699362041467305, 0.27435064935064934, 0.30980861244019137, 0.3807814992025519, 0.2760309865573024, 0.27870813397129185, 0.32396331738437, 0.30923900660742765, 0.2641831852358168, 0.27625882889040787, 0.2972488038277512], [0.59563681932103, 0.6668945089997721, 0.4877249943039417, 0.5193096377306904, 0.5034176349965824, 0.6207564365459103, 0.5433470038733197, 0.47114946457051715, 0.4752221462747779, 0.42720437457279564, 0.5280815675552518, 0.49962975620870353, 0.49988607883344727, 0.702323991797676, 0.566928685349738, 0.6258259284575074, 0.5502392344497608, 0.5849851902483482, 0.5497265892002734, 0.6576099339257233, 0.4838231943495101, 0.46610845295055825, 0.4609820004556846, 0.6011619958988381, 0.5361130097972203, 0.6249145591250854, 0.6001367053998633, 0.7082194121667805, 0.5466791979949874, 0.4425267714741399, 0.5710583276372748, 0.5292207792207793, 0.4909432672590567, 0.5037593984962406, 0.4090909090909091, 0.5942697653223967, 0.4632604237867396, 0.5335497835497836, 0.4665641376167692, 0.4039644565960355, 0.5670426065162907, 0.5185691501480976, 0.6991911597174754, 0.5239519252677147, 0.5593529277739804, 0.47305764411027573, 0.5156641604010025, 0.5878332194121668, 0.5298188653451812, 0.5477329687856004], [0.6700843016632492, 0.6241171109592163, 0.5897698792435635, 0.6941786283891547, 0.5518341307814991, 0.7087035771246296, 0.6847801321485532, 0.632490316700843, 0.585554796081112, 0.7081909318751424, 0.6887104123946229, 0.6604579630895421, 0.6200159489633175, 0.6593757120072908, 0.6609136477557531, 0.5795739348370927, 0.6488380041011621, 0.5256322624743678, 0.592162223741171, 0.6713374344953292, 0.6368762816131237, 0.6286739576213259, 0.6078263841421736, 0.6054910002278423, 0.6466165413533834, 0.6003075871496925, 0.6438254727728411, 0.6523125996810207, 0.6019024834814309, 0.7112098427887901, 0.773524720893142, 0.6384711779448622, 0.5285942128047392, 0.7079630895420369, 0.5929596719070402, 0.7200387331966279, 0.6504329004329005, 0.6560719981772613, 0.6458760537707906, 0.7572909546593757, 0.5745044429254955, 0.6699703804966963, 0.5907951697425381, 0.7167919799498746, 0.5989974937343358, 0.5793460925039873, 0.6208133971291867, 0.5685805422647529, 0.7729551150603782, 0.6590909090909092], [0.6484962406015038, 0.6293574846206427, 0.6231487810435179, 0.5853269537480064, 0.6270221007063113, 0.5737639553429026, 0.5543973570289359, 0.5858965595807702, 0.4941330599225336, 0.6303827751196173, 0.5837890179995443, 0.6209273182957393, 0.5734791524265209, 0.578092959671907, 0.4479380268853953, 0.6392686261107314, 0.5441729323308271, 0.5738209159261791, 0.43483709273182963, 0.6014467988152198, 0.4784688995215311, 0.5459102301207565, 0.6326042378673957, 0.6130667578035999, 0.5975734791524265, 0.6094212804739121, 0.588174982911825, 0.6012759170653907, 0.5735930735930737, 0.5289074960127591, 0.587092731829574, 0.6060036454773297, 0.533777625882889, 0.5848712690817954, 0.577010708589656, 0.41960013670539986, 0.697881066302119, 0.4889496468443837, 0.606516290726817, 0.5421508316245157, 0.5310435178856232, 0.5273980405559353, 0.5837320574162679, 0.5059808612440191, 0.49624060150375937, 0.5813397129186603, 0.6576099339257233, 0.5856687172476646, 0.44736842105263164, 0.607142857142857], [0.49316473000683525, 0.550580997949419, 0.4266347687400319, 0.5134426976532239, 0.568010936431989, 0.4607541581225792, 0.38402825244930505, 0.4580200501253133, 0.5243791296422875, 0.5787195260879472, 0.5756436545910231, 0.42484051036682613, 0.5630553656869446, 0.6105320118478013, 0.44668489405331513, 0.538049669628617, 0.4677318295739349, 0.5670995670995671, 0.5819662793347002, 0.3948507632718159, 0.5672134882661197, 0.5364547732968785, 0.4821713374344954, 0.507632718159034, 0.5663875598086124, 0.4458874458874459, 0.5011961722488039, 0.5997379813169286, 0.4783549783549783, 0.5405274550011392, 0.4778708133971292, 0.5089997721576669, 0.5395021645021644, 0.530872636135794, 0.6173957621326043, 0.45169742538163593, 0.5341763499658237, 0.36933242196400096, 0.48547505126452495, 0.5983709273182958, 0.5356573251310094, 0.49248120300751885, 0.507632718159034, 0.46793119161540214, 0.6258259284575074, 0.6770334928229665, 0.4392800182273866, 0.6450786056049215, 0.4839371155160629, 0.5182273866484393], [0.15567327409432674, 0.18213146502620184, 0.13998063340168604, 0.1764069264069264, 0.2532467532467533, 0.17603668261563, 0.1251993620414673, 0.1923274094326726, 0.1877420824789246, 0.14433811802232854, 0.2807871952608795, 0.20064365459102304, 0.1179084073820916, 0.19434951013898383, 0.1404647983595352, 0.16689450899977215, 0.18586238323080428, 0.1988493962178173, 0.14456596035543406, 0.15860674413305992, 0.1730177717019822, 0.16660970608339032, 0.26896787423103213, 0.2502278423331055, 0.16398951925267716, 0.18648894964684437, 0.15669856459330145, 0.20186830713146503, 0.18267259056732743, 0.08942811574390522, 0.18258714969241285, 0.2363864205969469, 0.21915584415584413, 0.16296422875370245, 0.18449532923217138, 0.25079744816586924, 0.1482683982683983, 0.14035087719298248, 0.14408179539758487, 0.24567099567099568, 0.17321713374344955, 0.15521758942811578, 0.17170767828662564, 0.191928685349738, 0.14325586694007747, 0.11181362497151971, 0.21360218728639782, 0.18910913647755753, 0.15749601275917063, 0.11881977671451356], [0.6369902027796764, 0.6499202551834131, 0.5702893597630441, 0.3456083390293917, 0.6179084073820915, 0.5918774208247892, 0.5742766005923902, 0.5668147641831852, 0.5249487354750513, 0.567555251765778, 0.4899749373433584, 0.552118933697881, 0.4249829118250171, 0.5326953748006379, 0.4820004556846662, 0.5836181362497153, 0.44765322396901347, 0.4344953292321714, 0.4991455912508544, 0.5148667122351334, 0.5328092959671906, 0.5645933014354066, 0.548758259284575, 0.47419685577580317, 0.45933014354066987, 0.5557644110275689, 0.47715880610617456, 0.5475620870357711, 0.4656527682843472, 0.6016746411483254, 0.59107997265892, 0.6331168831168831, 0.6059466848940533, 0.6293005240373662, 0.48165869218500795, 0.5940419229892914, 0.7367281840966051, 0.5055821371610845, 0.456596035543404, 0.5008544087491457, 0.5698336750968329, 0.456596035543404, 0.5380496696286169, 0.6659831396673502, 0.48325358851674644, 0.40498974709501023, 0.3873604465709729, 0.5517771701982227, 0.5986557302346776, 0.5637958532695374], [0.2799043062200957, 0.3038277511961722, 0.270619731146047, 0.30137844611528825, 0.4616655274550011, 0.3239348370927319, 0.384313055365687, 0.3122009569377991, 0.48798131692868535, 0.38590795169742537, 0.4152996126680337, 0.3508771929824561, 0.4718329915698337, 0.2382661198450672, 0.33347573479152426, 0.24111414900888584, 0.3147072226019595, 0.3224823422191843, 0.4504442925495557, 0.35127591706539074, 0.4331282752335384, 0.3691615402141718, 0.3697596263385737, 0.4019138755980861, 0.4592162223741172, 0.24883230804283438, 0.28406242879927096, 0.2976475279106858, 0.3376908179539758, 0.4972374117110958, 0.3071884256094783, 0.30217589428115743, 0.40686944634313055, 0.4719753930280246, 0.499088630667578, 0.3507063112326271, 0.41615402141717933, 0.34666210982000456, 0.4481089086352244, 0.2706766917293233, 0.4511278195488722, 0.3468899521531101, 0.3741740715424926, 0.29784688995215314, 0.28918888129414444, 0.4226475279106858, 0.33373205741626794, 0.3157894736842105, 0.3385737069947597, 0.2888755980861245], [0.27905540978546073, 0.32532026547306686, 0.3694242938725112, 0.34820188300663685, 0.33215002315172093, 0.3250501620620466, 0.3067603025158203, 0.3149405772495756, 0.33747491896897674, 0.328985954622627, 0.27172403148634044, 0.34195091835159747, 0.3732829140299429, 0.32304367958018215, 0.3394042290476926, 0.3184905077944127, 0.26805834233678033, 0.3830838092298194, 0.2564438956629109, 0.4292329063127026, 0.2904769254514586, 0.2594536193857077, 0.35468436487112215, 0.394968359314709, 0.28237382312085196, 0.37991974070072543, 0.34272264238308375, 0.29016823583886403, 0.2722256521068066, 0.29425837320574166, 0.3357771260997067, 0.319493749035345, 0.4389566291094305, 0.38921901528013586, 0.2402762772032721, 0.2296650717703349, 0.2995060966198487, 0.3238925760148171, 0.34642691773421824, 0.3881000154344806, 0.3448834696712456, 0.3809229819416577, 0.2677496527241858, 0.2857308226578176, 0.3631347430158975, 0.2458712764315481, 0.24062355301744096, 0.33060657508874824, 0.3891804290785615, 0.29746102793641], [0.4600136705399864, 0.5666438824333561, 0.4612098427887902, 0.42543859649122806, 0.5349168375484166, 0.44036226930963773, 0.6377876509455456, 0.6404078377762588, 0.558384597858282, 0.5401002506265664, 0.481203007518797, 0.37451583504215086, 0.5539986329460013, 0.46274777853725224, 0.5369674185463659, 0.5592959671907041, 0.48752563226247436, 0.5583276372750057, 0.5320118478013216, 0.6446798815219869, 0.5612326270221007, 0.459045340624288, 0.6504329004329004, 0.571827295511506, 0.4027113237639553, 0.6236044657097288, 0.45494417862838915, 0.5694349510138983, 0.5464798359535201, 0.6289587605377079, 0.4568808384597858, 0.5794030530872637, 0.6136933242196402, 0.521701982228298, 0.43962178172704497, 0.4461722488038277, 0.5378787878787878, 0.4797220323536113, 0.6273638642059696, 0.5747892458418775, 0.5666438824333561, 0.5775233538391434, 0.6008202323991798, 0.5811688311688311, 0.5220437457279562, 0.6579516974253816, 0.6393255866940079, 0.5200501253132832, 0.5845295055821371, 0.5565618591934381], [0.08199475962633859, 0.06561859193438141, 0.08387445887445888, 0.04192298929141036, 0.08837434495329233, 0.09691843244474822, 0.055422647527910686, 0.05738778765094555, 0.10042150831624518, 0.04243563454089771, 0.08583959899749374, 0.06009341535657325, 0.12030075187969926, 0.05223285486443381, 0.05724538619275462, 0.072738664843928, 0.07678286625655045, 0.11386420596946915, 0.08498519024834815, 0.08689336978810665, 0.07091592617908409, 0.07177033492822968, 0.05844155844155844, 0.0800865800865801, 0.07450444292549557, 0.08908635224424699, 0.09483937115516064, 0.029733424470266584, 0.07931761221234905, 0.06351105035315563, 0.055080884028252446, 0.04536910457963089, 0.07399179767600822, 0.09261790840738211, 0.09327295511506038, 0.09498177261335157, 0.07040328092959673, 0.12756322624743677, 0.09079516974253816, 0.04796081111870586, 0.0873775347459558, 0.0938995215311005, 0.05206197311460469, 0.03232513100934154, 0.03520164046479836, 0.12531328320802007, 0.06567555251765778, 0.03722374117110959, 0.08182387787650945, 0.10378218272955114], [0.26953748006379585, 0.3920881749829118, 0.300580997949419, 0.3898382319434951, 0.30556504898610165, 0.4469127363864206, 0.30362838915470497, 0.4069833675096833, 0.2756892230576441, 0.32840624287992704, 0.3494531784005469, 0.34637730690362273, 0.41048644338118023, 0.35415242652084755, 0.36562998405103664, 0.39462292093871043, 0.4679596719070403, 0.377136021872864, 0.4294827979038506, 0.326697425381636, 0.39154704944178625, 0.37810435178856233, 0.4398496240601503, 0.32595693779904306, 0.2971633629528367, 0.31473570289359765, 0.30379927090453407, 0.37497151970836184, 0.3904078377762588, 0.4662793347003873, 0.4053599908863067, 0.3178970152654363, 0.20457393483709274, 0.36719640009113697, 0.35173160173160173, 0.41541353383458646, 0.30291638186375025, 0.28605604921394395, 0.4162109820004557, 0.3427318295739349, 0.3346149464570517, 0.3957051720209615, 0.2677147413989519, 0.2461266803372067, 0.3849396217817271, 0.3478582820688084, 0.3684210526315789, 0.501680337206653, 0.34398496240601506, 0.4516404647983595], [0.626224652540442, 0.7263043973570289, 0.6862611073137389, 0.6939507860560492, 0.6042378673957621, 0.647015265436318, 0.7920369104579631, 0.6727614490772386, 0.8198906356801093, 0.5862952836637046, 0.7089314194577352, 0.7542150831624514, 0.6987924356345409, 0.6605718842560948, 0.7038049669628617, 0.6143198906356802, 0.7285828206880838, 0.721120984278879, 0.7468102073365231, 0.7109250398724083, 0.7255639097744361, 0.7299498746867168, 0.6354522670312144, 0.7128047391205286, 0.5809979494190021, 0.849794941900205, 0.5356573251310094, 0.7403167008430166, 0.574105718842561, 0.7240829346092504, 0.6086807928913193, 0.6826725905673274, 0.7253360674413306, 0.6642173615857826, 0.672077922077922, 0.7211779448621555, 0.6126110731373889, 0.691786283891547, 0.6435406698564593, 0.6802802460697197, 0.5446001367053999, 0.6672362724994305, 0.8397129186602872, 0.7259056732740944, 0.7323991797676007, 0.5692071086807929, 0.7345067213488267, 0.729209387104124, 0.5462519936204147, 0.5644793802688539]]
all_auc = [[0.5783208020050125, 0.6294714057871953, 0.6302688539530643, 0.6134085213032582, 0.61369332421964, 0.6209273182957394, 0.5541410344041923, 0.5239519252677147, 0.6011050353155617, 0.6261107313738893, 0.5618022328548645, 0.5706311232627022, 0.7018683071314651, 0.56940647072226, 0.5529733424470266, 0.7162793347003873, 0.5990544543176122, 0.5274265208475735, 0.5669002050580998, 0.5980291638186375, 0.6132946001367054, 0.619958988380041, 0.6467874231032126, 0.6439393939393939, 0.6623661426293006, 0.5397015265436318, 0.5879756208703577, 0.6143198906356802, 0.537593984962406, 0.5918204602415129, 0.654021417179312, 0.6121553884711779, 0.5644224196855776, 0.6309523809523809, 0.6279334700387331, 0.6501480975165186, 0.6829573934837093, 0.507177033492823, 0.4242709045340624, 0.6688311688311688, 0.666097060833903, 0.5186261107313739, 0.6060036454773297, 0.5819662793347004, 0.5937856003645477, 0.5941558441558442, 0.5652768284347232, 0.6042378673957621, 0.6561289587605377, 0.6211266803372066], [0.1066302118933698, 0.15006265664160404, 0.0944691273638642, 0.08515607199817726, 0.14835383914331282, 0.1640749601275917, 0.14661654135338348, 0.07983025746183643, 0.10563340168603327, 0.06635907951697426, 0.10600364547732971, 0.1331738437001595, 0.09936773752563227, 0.10275689223057645, 0.13385737069947595, 0.15578719526087947, 0.10247208931419458, 0.17825814536340853, 0.11397812713602189, 0.08564023695602643, 0.13687628161312373, 0.13847117794486216, 0.053941672362725006, 0.12109820004556847, 0.16307814992025516, 0.15026201868307132, 0.10124743677375256, 0.10554796081111871, 0.14177489177489175, 0.06596035543403965, 0.10748462064251538, 0.15937571200729095, 0.06613123718386876, 0.12121212121212124, 0.10426634768740031, 0.1095352016404648, 0.13610731373889268, 0.12047163362952838, 0.1496924128503076, 0.12138300296195033, 0.08669400774663934, 0.10697197539302804, 0.15829346092503987, 0.10235816814764184, 0.07060264297106403, 0.10529163818637502, 0.11130097972203236, 0.16752107541581227, 0.16205285942128048, 0.08037138300296195], [0.5354294827979038, 0.5112212349054455, 0.546764638869902, 0.3885281385281386, 0.5697767145135566, 0.457222601959444, 0.3522442469810891, 0.3833447254499886, 0.485930735930736, 0.5086010480747323, 0.5264866712235133, 0.3841991341991342, 0.543233082706767, 0.5285372522214626, 0.5098541809068124, 0.46758942811574394, 0.5294486215538847, 0.4598997493734336, 0.5146388699020278, 0.5459102301207565, 0.4943039416723627, 0.4346662109820005, 0.422989291410344, 0.3771929824561403, 0.5906812485759854, 0.43375484164957856, 0.3457507404875826, 0.5250056960583277, 0.4297676008202324, 0.5521758942811574, 0.37844611528822053, 0.6488380041011621, 0.49720893141945777, 0.5283094098883572, 0.4880952380952381, 0.49504442925495556, 0.4289131920710868, 0.429254955570745, 0.527511961722488, 0.31823877876509454, 0.46160856687172475, 0.39194577352472093, 0.4355775803144224, 0.5734221918432445, 0.5630553656869446, 0.5668147641831852, 0.43711551606288446, 0.4348940533151059, 0.5199931647300068, 0.4190590111642743], [0.25313283208020054, 0.2832934609250399, 0.2896445659603555, 0.250911369332422, 0.3293460925039872, 0.2780246069719754, 0.24877534745955798, 0.2602813852813853, 0.28252449305080884, 0.3617281840966051, 0.29625199362041466, 0.24025974025974028, 0.3397983595352017, 0.31647300068352696, 0.30063795853269537, 0.301492367281841, 0.2813852813852814, 0.3203463203463204, 0.19076099339257235, 0.25353155616313516, 0.29579630895420367, 0.20821941216678058, 0.22274436090225563, 0.25945545682387783, 0.26253132832080206, 0.3677660059239007, 0.27321143768512185, 0.3002677147413989, 0.20055821371610846, 0.2844611528822055, 0.23650034176349966, 0.21892800182273867, 0.19574504442925497, 0.2642401458190932, 0.3200615174299385, 0.19762474367737526, 0.2575187969924812, 0.2654647983595352, 0.24216791979949878, 0.2218045112781955, 0.23957621326042375, 0.2602813852813853, 0.34529505582137154, 0.3929710640236957, 0.2589712918660287, 0.24071542492595124, 0.22761449077238552, 0.2923786739576214, 0.14809751651856914, 0.35440874914559134], [0.5431191615402142, 0.5984848484848484, 0.5651629072681704, 0.536825017088175, 0.4389952153110048, 0.5619161540214173, 0.46348826611984506, 0.5415242652084757, 0.5813397129186603, 0.5383344725449989, 0.45807701070858964, 0.49720893141945777, 0.44543176122123485, 0.5202779676463887, 0.5802574618364092, 0.42367281840966053, 0.45101389838231937, 0.5103098655730235, 0.581453634085213, 0.5055821371610846, 0.5356573251310093, 0.5706311232627022, 0.5389610389610389, 0.4370585554796081, 0.4915698336750968, 0.5712861699703806, 0.6326042378673957, 0.6306675780359992, 0.5101105035315562, 0.45215311004784686, 0.4952153110047846, 0.442697653223969, 0.5484164957849168, 0.5758714969241285, 0.42976760082023246, 0.6255980861244019, 0.6385850991114149, 0.4322738664843928, 0.5234677603098655, 0.5086010480747323, 0.6618250170881751, 0.4169514695830486, 0.5824219640009113, 0.5363123718386876, 0.6017316017316017, 0.6198450672134882, 0.49846206425153794, 0.6536226930963772, 0.5793460925039873, 0.5911938938254728], [0.5943836864889496, 0.6168261562998405, 0.6519708361813625, 0.5259170653907496, 0.6408635224424699, 0.5709728867623605, 0.6014467988152199, 0.5727956254272044, 0.6917862838915471, 0.6528822055137845, 0.5841877420824789, 0.5439166097060835, 0.5783777625882889, 0.6748689906584643, 0.5944406470722261, 0.6283321941216677, 0.7168489405331511, 0.6210412394622921, 0.5757006151742994, 0.592276144907724, 0.5667008430166324, 0.6467874231032125, 0.47818409660514927, 0.5672704488493963, 0.7042036910457964, 0.667350193665983, 0.6191615402141718, 0.49934495329232176, 0.6069719753930279, 0.4853611300979722, 0.5616883116883118, 0.7267600820232399, 0.49988607883344727, 0.5951811346548188, 0.5977443609022557, 0.5789473684210527, 0.5675552517657781, 0.5734221918432444, 0.4730006835269993, 0.6811346548188654, 0.6170539986329461, 0.6880838459785829, 0.5693210298473458, 0.6161426293005241, 0.631578947368421, 0.7180451127819549, 0.6739576213260423, 0.6224652540442014, 0.6967418546365916, 0.7010138983823194], [0.6157439052175895, 0.7031784005468217, 0.5853839143312827, 0.7118364092048303, 0.5419799498746867, 0.6722488038277512, 0.5578149920255183, 0.6296992481203008, 0.5176577808156756, 0.6633059922533607, 0.6441672362724994, 0.6425723399407609, 0.5747322852586011, 0.6060036454773298, 0.6061745272271589, 0.5737069947596264, 0.6768626110731374, 0.634028252449305, 0.7339940760993392, 0.6053201184780131, 0.5942128047391205, 0.41820460241512875, 0.5350877192982456, 0.6058327637275005, 0.602529049897471, 0.6367053998632946, 0.5658464342674869, 0.6596605149236727, 0.6070858965595808, 0.7325700615174299, 0.633401686033265, 0.5603212576896788, 0.5725108225108225, 0.4700387331966279, 0.6731601731601732, 0.5416381863750286, 0.49082934609250406, 0.5677261335156072, 0.5123604465709729, 0.6985076327181591, 0.6932672590567328, 0.5573593073593075, 0.48986101617680566, 0.5751310093415356, 0.628673957621326, 0.49054454317612206, 0.5169742538163591, 0.7355320118478014, 0.6679767600820233, 0.568295739348371], [0.6008771929824561, 0.5886876281613124, 0.5967760309865574, 0.47003873319662787, 0.4339542036910458, 0.4776144907723855, 0.5095693779904306, 0.7141148325358851, 0.4301093643198906, 0.6804511278195489, 0.561574390521759, 0.5222146274777854, 0.555622009569378, 0.6371041239462292, 0.6327466393255867, 0.5965481886534518, 0.5019936204146731, 0.6729038505354296, 0.5228411938938254, 0.43067897015265433, 0.4496468443836865, 0.5956937799043062, 0.5287650945545682, 0.5607769423558897, 0.4750512645249487, 0.5069776714513556, 0.37380382775119614, 0.5317270448849396, 0.4801777170198223, 0.5006265664160401, 0.7322282980177717, 0.6888812941444521, 0.5596946912736387, 0.5195374800637959, 0.5019366598313967, 0.5587833219412167, 0.5135566188197768, 0.49700956937799046, 0.5915356573251309, 0.6787423103212576, 0.5522898154477103, 0.5337206652996127, 0.5976874003189793, 0.5653337890179996, 0.6216678058783323, 0.4996582365003418, 0.6021303258145364, 0.5668717247664616, 0.6098200045568467, 0.6195602642971064], [0.17805878332194122, 0.10982000455684666, 0.14103440419229896, 0.07245386192754613, 0.2506835269993165, 0.17737525632262477, 0.188112326270221, 0.14869560264297108, 0.11944634313055368, 0.1467304625199362, 0.14946457051720208, 0.10007974481658694, 0.15031897926634769, 0.13411369332421963, 0.1695716564137617, 0.11964570517202097, 0.12075643654591026, 0.15111642743221693, 0.19064707222601962, 0.1776600592390066, 0.16678058783321942, 0.1887958532695375, 0.12075643654591026, 0.1627648667122351, 0.12092731829573936, 0.12371838687628162, 0.1279334700387332, 0.20135566188197768, 0.19953292321713373, 0.15043290043290045, 0.10982000455684666, 0.15983139667350194, 0.1290157211209843, 0.11836409204830257, 0.10532011847801323, 0.12685121895648213, 0.11409204830257462, 0.2296081111870586, 0.15817953975848714, 0.1651002506265664, 0.2197539302802461, 0.11967418546365914, 0.09845636819321031, 0.15903394850763272, 0.18515037593984962, 0.24419002050580998, 0.12377534745955798, 0.16518569150148094, 0.12679425837320574, 0.18674527227158805], [0.5278537252221464, 0.5739348370927317, 0.5735361130097971, 0.445289359763044, 0.5928457507404876, 0.5571884256094782, 0.4469127363864206, 0.5923615857826384, 0.4550011392116655, 0.6119845067213489, 0.5235816814764183, 0.5944976076555024, 0.509626338573707, 0.6589769879243563, 0.5248917748917749, 0.5521189336978811, 0.4821713374344953, 0.5494417862838916, 0.5112781954887219, 0.6791979949874686, 0.5218728639781272, 0.5213032581453634, 0.6494076099339258, 0.5262588288904079, 0.5330940988835726, 0.4914559125085441, 0.558669400774664, 0.5686944634313055, 0.5747322852586011, 0.5448279790385054, 0.6671223513328777, 0.5918774208247892, 0.6371610845295055, 0.5911369332421964, 0.48046251993620415, 0.41421736158578265, 0.4675324675324675, 0.48934837092731837, 0.49760765550239233, 0.4702665755297334, 0.5583845978582821, 0.5826498063340169, 0.5641945773524721, 0.6375598086124402, 0.5189109136477558, 0.6006493506493507, 0.6295283663704716, 0.47755753018910907, 0.7008999772157667, 0.5479608111187059], [0.36010480747322854, 0.48860788334472544, 0.5350592390066075, 0.3168147641831852, 0.431989063568011, 0.32424812030075184, 0.38664843928001824, 0.3403679653679654, 0.4554568238778765, 0.33988380041011623, 0.3226817042606517, 0.36101617680565046, 0.39678742310321263, 0.43079289131920717, 0.35594668489405334, 0.35252904989747097, 0.3628389154704945, 0.31265664160401, 0.35979152426520844, 0.3685064935064935, 0.3262132604237868, 0.4209387104123946, 0.3258145363408521, 0.29206539074960125, 0.38804397357028936, 0.35574732285258603, 0.3562314878104352, 0.29536910457963095, 0.37924356345408977, 0.409688995215311, 0.31732740943267257, 0.3984392800182274, 0.40960355434039647, 0.5925039872408293, 0.36654135338345867, 0.25996810207336524, 0.3274094326725906, 0.33629528366370476, 0.37756322624743677, 0.2496582365003418, 0.3953064479380269, 0.30285942128047394, 0.3507632718159034, 0.4552005012531328, 0.3490259740259741, 0.47300068352699937, 0.389211665527455, 0.38972431077694236, 0.375768967874231, 0.3402825244930508], [0.23471986417657048, 0.3239697484179658, 0.32751967896280293, 0.28002006482481867, 0.2904769254514586, 0.39527704892730364, 0.28164068529093994, 0.2408550702268869, 0.31629109430467667, 0.25212224108658743, 0.3681123630189844, 0.270566445439111, 0.27585275505479245, 0.4209754591757988, 0.3642151566599784, 0.3594690538663374, 0.4756135206050316, 0.25872048155579563, 0.3095770952307455, 0.33670319493749035, 0.3427226423830838, 0.34233678036734067, 0.30525544065442195, 0.2742321345886711, 0.18563821577403922, 0.43343880228430315, 0.36911560425991663, 0.30741626794258375, 0.37582960333384785, 0.4159592529711375, 0.21824355610433713, 0.1866028708133971, 0.38154036116684675, 0.32057416267942584, 0.3404074702886248, 0.3391727118382466, 0.39921284148788394, 0.3148248186448526, 0.37004167309770025, 0.35302515820342645, 0.3882543602407779, 0.3754437413181047, 0.3745176724803211, 0.36726346658434944, 0.39365642846118226, 0.2746951690075629, 0.27731903071461644, 0.25096465503935794, 0.2567525852755055, 0.3417194011421516], [0.5656185919343815, 0.4628616997038049, 0.6535087719298247, 0.5142401458190932, 0.4537764866712235, 0.599168375484165, 0.6424584187742082, 0.6263955342902711, 0.5516062884483937, 0.6425153793574846, 0.6421736158578264, 0.7416267942583731, 0.5712007290954659, 0.39673046251993616, 0.711038961038961, 0.6585213032581453, 0.5514354066985646, 0.5847573479152427, 0.6240031897926634, 0.48080428343586235, 0.5780929596719069, 0.4566529961266803, 0.5461380724538619, 0.4660514923672819, 0.5763841421736159, 0.5926748689906585, 0.5763841421736158, 0.5234107997265892, 0.5951811346548189, 0.5233538391433128, 0.4448621553884712, 0.5836750968329916, 0.588915470494418, 0.6618250170881749, 0.6061175666438825, 0.628673957621326, 0.5607199817726134, 0.566928685349738, 0.5868364092048303, 0.5425495557074506, 0.5369104579630894, 0.48094668489405323, 0.5082592845750741, 0.7312030075187971, 0.5436887673729779, 0.5446570972886762, 0.6346548188653452, 0.701412622465254, 0.6391547049441787, 0.6522556390977443], [0.04123946229209388, 0.041068580542264754, 0.12383230804283436, 0.12289245841877422, 0.0773524720893142, 0.03765094554568239, 0.04722032353611301, 0.06111870585554796, 0.08680792891319208, 0.08544087491455912, 0.05425495557074504, 0.14519252677147415, 0.053713830029619505, 0.1022442469810891, 0.07370699475962635, 0.07444748234221918, 0.07105832763727501, 0.06308384597858283, 0.12115516062884484, 0.09079516974253816, 0.06447938026885396, 0.08418774208247892, 0.10483595352016405, 0.10076327181590339, 0.050751879699248124, 0.08341877420824789, 0.08749145591250855, 0.08071314650262021, 0.07792207792207793, 0.11568694463431306, 0.11107313738892687, 0.07262474367737526, 0.08695033037138301, 0.12593984962406016, 0.07911825017088175, 0.0917634996582365, 0.07635566188197766, 0.12642401458190933, 0.10597516518569151, 0.050096832991569834, 0.06721348826611986, 0.05758714969241285, 0.07293802688539532, 0.11637047163362954, 0.06801093643198908, 0.05445431761221235, 0.0524606971975393, 0.09580770107085897, 0.10053542948279791, 0.07179881521986786], [0.4092902711323764, 0.344127363864206, 0.2907268170426065, 0.3845978582820688, 0.45277967646388706, 0.36198450672134874, 0.4888072453861928, 0.3793859649122806, 0.35828206880838465, 0.406641604010025, 0.3327922077922078, 0.3094098883572568, 0.3508771929824561, 0.36916154021417175, 0.356516290726817, 0.3784461152882206, 0.4073820915926179, 0.28833447254499883, 0.3585953520164047, 0.25871496924128506, 0.3421622237411711, 0.41831852358168153, 0.32866256550467077, 0.5093415356573251, 0.3364092048302575, 0.4202836637047163, 0.4215652768284347, 0.40521758942811575, 0.2594554568238779, 0.3688482570061517, 0.3708418774208248, 0.3650318979266348, 0.4078947368421052, 0.3284347231715653, 0.44207108680792895, 0.356516290726817, 0.38784461152882205, 0.4368307131465026, 0.2838915470494418, 0.3440419229892915, 0.3839143312827523, 0.46394395078605605, 0.3937685121895648, 0.3881863750284803, 0.28058783321941216, 0.3823763955342903, 0.3076441102756892, 0.4162109820004557, 0.29158122579175216, 0.4551720209614946], [0.7750056960583277, 0.7627022100706312, 0.5477899293688767, 0.6263385737069949, 0.7054568238778764, 0.7323991797676008, 0.7063681932102985, 0.6464456596035544, 0.6355092276144907, 0.6377876509455457, 0.678343586238323, 0.699134199134199, 0.6255411255411256, 0.8205172020961494, 0.6536226930963773, 0.7076213260423787, 0.7159945317840055, 0.7060264297106401, 0.7676577808156756, 0.7345067213488266, 0.6728753702437913, 0.6860332649806333, 0.6210982000455685, 0.7659489633173843, 0.5715424925951241, 0.6191615402141718, 0.5813966735019367, 0.6897357028935976, 0.7177033492822966, 0.6442241968557758, 0.7838915470494418, 0.5852130325814536, 0.7603098655730234, 0.6333447254499888, 0.5412109820004557, 0.6234905445431762, 0.6580086580086579, 0.7272157666894509, 0.6937229437229439, 0.618705855547961, 0.6944634313055366, 0.6981658692185007, 0.6245727956254273, 0.5938140806561859, 0.6605718842560949, 0.5840738209159262, 0.6846092503987242, 0.6719640009113693, 0.5753588516746412, 0.775518341307815]]

[[0.5783208020050125, 0.6294714057871953, 0.6302688539530643, 0.6134085213032582, 0.61369332421964, 0.6209273182957394, 0.5541410344041923, 0.5239519252677147, 0.6011050353155617, 0.6261107313738893, 0.5618022328548645, 0.5706311232627022, 0.7018683071314651, 0.56940647072226, 0.5529733424470266, 0.7162793347003873, 0.5990544543176122, 0.5274265208475735, 0.5669002050580998, 0.5980291638186375, 0.6132946001367054, 0.619958988380041, 0.6467874231032126, 0.6439393939393939, 0.6623661426293006, 0.5397015265436318, 0.5879756208703577, 0.6143198906356802, 0.537593984962406, 0.5918204602415129, 0.654021417179312, 0.6121553884711779, 0.5644224196855776, 0.6309523809523809, 0.6279334700387331, 0.6501480975165186, 0.6829573934837093, 0.507177033492823, 0.4242709045340624, 0.6688311688311688, 0.666097060833903, 0.5186261107313739, 0.6060036454773297, 0.5819662793347004, 0.5937856003645477, 0.5941558441558442, 0.5652768284347232, 0.6042378673957621, 0.6561289587605377, 0.6211266803372066], [0.106

In [26]:
all_auc

[[0.616142629300524,
  0.6383572567783093,
  0.5963773069036227,
  0.6338004101161995,
  0.6377306903622694,
  0.603611300979722,
  0.5831624515835042,
  0.6329175210754159,
  0.7038049669628617,
  0.5924755069491912,
  0.6768626110731375,
  0.5813966735019367,
  0.6433128275233537,
  0.6401230348598769,
  0.6944064707222602,
  0.6308384597858283,
  0.6263385737069946,
  0.5777511961722488,
  0.6328320802005012,
  0.6513442697653224,
  0.6728753702437912,
  0.6473000683526999,
  0.5798587377534746,
  0.5986557302346777,
  0.5558213716108454,
  0.6254272043745729,
  0.6701982228298018,
  0.7304055593529278,
  0.6471861471861471,
  0.6431989063568011,
  0.7268740031897927,
  0.6901913875598086,
  0.5944976076555024,
  0.697425381635908,
  0.6835839598997494,
  0.617680565048986,
  0.7439052175894281,
  0.6066871724766462,
  0.5787764866712235,
  0.5733082706766917,
  0.6071428571428572,
  0.6940077466393255,
  0.578548644338118,
  0.5864376851218956,
  0.6512303485987696,
  0.65299612668

In [48]:
import seaborn as sns

numpy_array = np. array(all_auc)
transpose = numpy_array.T.tolist()
auc_df = pd.DataFrame(transpose, columns=all_body_sites)
auc_df

sns.boxplot(data=auc_df['UBERON:central vagina'])

<AxesSubplot:>

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mayar\anaconda3\lib\site-packages\IPython\core\formatters.py", line 222, in catch_format_error
    r = method(self, *args, **kwargs)
  File "C:\Users\mayar\anaconda3\lib\site-packages\IPython\core\formatters.py", line 339, in __call__
    return printer(obj)
  File "C:\Users\mayar\anaconda3\lib\site-packages\IPython\core\pylabtools.py", line 151, in print_figure
    fig.canvas.print_figure(bytes_io, **kw)
  File "C:\Users\mayar\anaconda3\lib\site-packages\matplotlib\backend_bases.py", line 2319, in print_figure
    result = print_method(
  File "C:\Users\mayar\anaconda3\lib\site-packages\matplotlib\backend_bases.py", line 1648, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\mayar\anaconda3\lib\site-packages\matplotlib\_api\deprecation.py", line 412, in wrapper
    addendum=(addendum + " " + deprecation_addendum) if addendum
  File "C:\Users\mayar\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 541, 

<Figure size 432x288 with 1 Axes>

In [28]:
'''Only skin
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
all_skin_of_nose = metadata[metadata['body_site'] == 'UBERON:skin of nose']['#SampleID']
feces = [sample for sample in all_feces if sample in final_data.columns]
skin_of_nose = [sample for sample in all_skin_of_nose if sample in  final_data.columns]


feces_data = final_data[feces] #418 samples

skin_of_nose_data = final_data[skin_of_nose] #203 samples

few_skin_of_nose_data = skin_of_nose_data.iloc[: ,-42:] #42 samples

inputIF = pd.concat([feces_data, few_skin_of_nose_data], axis=1) #42 samples out of 460 should be outliers
model = IsolationForest(n_estimators=100)
model.fit(inputIF.transpose())  # fit 100 trees
 
scores = model.decision_function(inputIF.transpose()) #higher score means more normal
sortedScores = sorted(scores)

# indices of anomalies (skin of nose) in sorted scores. higher score = higher indices = more normal !
anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:]]
print(anomalies_indices_in_sorted) # if algorithm is working ok, indices are expected to be low. in reality we get differernt indices, some are very high. 

results = [1 for i in range(418)]
more_res = [0 for i in range(42)]
y = results + more_res

# calculate area under the curve
roc_auc_score(y, scores) #around 0.35
'''

"Only skin\nall_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']\nall_skin_of_nose = metadata[metadata['body_site'] == 'UBERON:skin of nose']['#SampleID']\nfeces = [sample for sample in all_feces if sample in final_data.columns]\nskin_of_nose = [sample for sample in all_skin_of_nose if sample in  final_data.columns]\n\n\nfeces_data = final_data[feces] #418 samples\n\nskin_of_nose_data = final_data[skin_of_nose] #203 samples\n\nfew_skin_of_nose_data = skin_of_nose_data.iloc[: ,-42:] #42 samples\n\ninputIF = pd.concat([feces_data, few_skin_of_nose_data], axis=1) #42 samples out of 460 should be outliers\nmodel = IsolationForest(n_estimators=100)\nmodel.fit(inputIF.transpose())  # fit 100 trees\n \nscores = model.decision_function(inputIF.transpose()) #higher score means more normal\nsortedScores = sorted(scores)\n\n# indices of anomalies (skin of nose) in sorted scores. higher score = higher indices = more normal !\nanomalies_indices_in_sorted = [i for i in range(le

graph depth of outlier vs normal
mann whitney test SCIPY
bootstrap